# Evaluation code for competition

## Load libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

## Set variables

In [2]:
confirmed='confirmed'
deaths='deaths'
recovered='recovered'

data_dir='Track1_Submitted/'

## Load in data files

### Submitter (predicted)

In [3]:
unis = [x for x in os.listdir(data_dir) if '.DS' not in x]
submitter_dict = {}
for uni in unis:
    uni_dict = {}
    files = [x for x in os.listdir(data_dir+uni) if '.DS' not in x]
    cfile = [x for x in files if confirmed in x]
    dfile = [x for x in files if deaths in x]
    rfile = [x for x in files if recovered in x]
    if len(cfile)>0:
        uni_dict[confirmed] = pd.read_csv(data_dir+uni+'/'+cfile[0])
    if len(dfile)>0:
        uni_dict[deaths] = pd.read_csv(data_dir+uni+'/'+dfile[0])
    if len(rfile)>0:
        uni_dict[recovered] = pd.read_csv(data_dir+uni+'/'+rfile[0])
    submitter_dict[uni] = uni_dict

### JHU (true)

In [4]:
jhu_dict = {}
jhu_dict[confirmed] = pd.read_csv('data/time_series_covid19_confirmed_global.csv')
jhu_dict[recovered] = pd.read_csv('data/time_series_covid19_recovered_global.csv')
jhu_dict[deaths] = pd.read_csv('data/time_series_covid19_deaths_global.csv')

## Comparing data files

In [7]:
true_dte = '4/29/20'
pred_dte = '5/1/20'
results_dict = {}
types = [confirmed,recovered,deaths]
for uni in submitter_dict.keys():
    if uni!='sl4646':
        print(uni)
        uni_dict = {}
        for type_ in types:
            df_s = submitter_dict[uni][type_].set_index(['Province/State','Country/Region'])
            df_j = jhu_dict[type_].set_index(['Province/State','Country/Region'])
            compare_df = \
            (df_j[[true_dte]].
             #convert an infinite values to NA
             replace([np.inf, -np.inf], np.nan).
             #drop NAs
             dropna().
             rename(columns = {true_dte : 'true'}).
             #only view countries/states with true counts>100
             query('true>100').
             join(df_s[[pred_dte]].
                   rename(columns = {pred_dte : 'pred'})
                 ).
             #convert any negative number to 1
             clip(lower=0).
             #convert 0 values to 1 (log(1) is 0)
             replace(0,1).
             replace([np.inf, -np.inf], np.nan).
             replace([np.nan, -np.nan],1).
             astype(np.float).
             apply(np.log)
            )
            y_pred = compare_df['pred'].values
            y_true = compare_df['true'].values
            mse = mean_squared_error(y_true,y_pred) ** 0.5
            uni_dict[type_] = mse
        results_dict[uni] = uni_dict

manasi.sharma
dlm2202
mcg2208
a.saakyan
by2287


TypeError: '<' not supported between instances of 'float' and 'str'

## Display RMSE ranking

In [36]:
pd.DataFrame(results_dict)

,manasi.sharma,dlm2202,mcg2208,a.saakyan
confirmed,0.051084,0.016686,0.020661,0.197797
deaths,0.024498,0.026546,0.031286,0.073239
recovered,5.226643,0.075114,0.076272,0.344621
